In [ ]:
import nest_asyncio
nest_asyncio.apply()

import aiohttp
import asyncio
import json
import pandas as pd
from datetime import datetime  # Change the import statement

# Define global variables for dataframes
student_df = None
engagement_df = None
ums_att_df = None
ums_avg_att_df = None

# Edoofy app information
edoofy_base_url = "https://edoofa-portal.bubbleapps.io/api/1.1/obj"
edoofy_bearer_token = "2cde31d8f48919a2db1467cc06a56132"
edoofy_headers = {'Authorization': f'Bearer {edoofy_bearer_token}'}

# UMS app information
ums_base_url = "https://edoofa-ums-90164.bubbleapps.io/version-test/api/1.1/obj"
ums_bearer_token = "786720e8eb68de7054d1149b56cc04f9"
ums_headers = {'Authorization': f'Bearer {ums_bearer_token}'}

# Asynchronous function to fetch data from a table
async def fetch_table_data(session, base_url, headers, table, constraints=None):
    records = []
    cursor = 0
    total_fetched = 0

    while True:
        params = {'limit': 100, 'cursor': cursor}
        if constraints:
            params['constraints'] = json.dumps(constraints)

        api_url = f"{base_url}/{table}"
        print(f"Fetching {table} data from {base_url}... Cursor: {cursor}")

        async with session.get(api_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch data from {table}: {await response.text()}")
                break

            data = await response.json()
            new_records = data['response']['results']
            records.extend(new_records)
            total_fetched += len(new_records)

            print(f"Fetched {len(new_records)} new records, Total fetched: {total_fetched}")

            cursor += 100

            if len(new_records) < 100:
                print(f"Exiting loop, fetched less than 100 records.")
                break

    df = pd.DataFrame(records)
    print(f"Fetched {len(df)} records for {table}.")
    return df

# Function to map students from Edoofy to Engagement dataframes
def map_students_to_engagement(student_df, engagement_df):
    # Create a dictionary to map student _id to EWYL-group-name
    student_mapping = dict(zip(student_df['_id'], student_df['EWYL-group-name']))

    # Map _id in engagement_df to EWYL-group-name
    engagement_df['student'] = engagement_df['student'].map(student_mapping)


# Usage example
async def main():
    global student_df, engagement_this_month_df, engagement_previous_months_df, ums_att_df, ums_avg_att_df
    
    async with aiohttp.ClientSession() as session:
        # Fetch Student table from Edoofy without constraints
        student_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Student")

        # Calculate the start and end date for the last 12 months (excluding this month)
        current_date = datetime.now()  # Use datetime without the "datetime" module
        end_date = current_date.replace(day=1) - timedelta(days=1)  # First day of the current month minus one day
        start_date = end_date - timedelta(days=365)  # Start date is 12 months before the end date

        # Create constraints for engagement dataframe for this month
        this_month_start = current_date.replace(day=1)
        this_month_end = current_date
        engagement_this_month_constraints = [
            {'key': 'engagement-date', 'constraint_type': 'greater than or equal', 'value': this_month_start.isoformat()},
            {'key': 'engagement-date', 'constraint_type': 'less than or equal', 'value': this_month_end.isoformat()},
            {'key': 'student', 'constraint_type': 'equals', 'value': '1695736497533x818400363201798900'}
        ]

        # Create constraints for engagement dataframe for previous months
        engagement_previous_months_constraints = [
            {'key': 'engagement-date', 'constraint_type': 'greater than', 'value': start_date.isoformat()},
            {'key': 'engagement-date', 'constraint_type': 'less than', 'value': this_month_start.isoformat()},
            {'key': 'student', 'constraint_type': 'equals', 'value': '1695736497533x818400363201798900'}
        ]

        # Fetch Engagement table from Edoofy with constraints for this month
        engagement_this_month_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Engagement", constraints=engagement_this_month_constraints)

        # Fetch Engagement table from Edoofy with constraints for previous months
        engagement_previous_months_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Engagement", constraints=engagement_previous_months_constraints)
        
        # Map students to engagement dataframes
        map_students_to_engagement(student_df, engagement_this_month_df)
        map_students_to_engagement(student_df, engagement_previous_months_df)
        
        
        # Fetch tables from UMS without constraints
        ums_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance")
        ums_avg_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance-Summary")

# Call the main function
await main()

# Now you can access the dataframes globally
print(student_df.head())  # Access Student dataframe
print(engagement_this_month_df.head())  # Access Engagement dataframe for this month
print(engagement_previous_months_df.head())  # Access Engagement dataframe for previous months
print(ums_att_df.head())  # Access UMS Attendance dataframe
print(ums_avg_att_df.head())  # Access UMS Average Attendance dataframe


Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 0
Fetched 100 new records, Total fetched: 100
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 100
Fetched 100 new records, Total fetched: 200
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 200
Fetched 100 new records, Total fetched: 300
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 300
Fetched 100 new records, Total fetched: 400
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 400
Fetched 100 new records, Total fetched: 500
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 500
Fetched 100 new records, Total fetched: 600
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 600
Fetched 100 new records, Total fetched: 700
Fetching Student data from https://edoofa-portal.bubbleapps.io/a